## Data Analysis Question: How are customers defecting?

Data Source: kaggle.com

Data Analyst: Alex Idachaba

In [366]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import datetime

In [367]:
# Load data
churn = pd.read_csv('churn.csv')
churn.head(3)

,image,newacct_no,line_stat,bill_cycl,serv_type,serv_code,tenure,effc_strt_date,effc_end_date,contract_month,ce_expiry,secured_revenue,bandwidth,term_reas_code,term_reas_desc,complaint_cnt,with_phone_service,churn,current_mth_churn,Unnamed: 19
0,201801,70068143.001.000000062,AC,1,BBS,VS100DUN,202,06.03.2018 00:00:00,05.03.2021 00:00:00,36.0,13.0,178.0,100M,NaN,NaN,0,N,N,N,NaN
1,201801,70068143.003.000072630,AC,1,BBS,VS030F6R,35,01.03.2019 00:00:00,28.02.2021 00:00:00,24.0,13.0,1572.0,30M,NaN,NaN,0,N,N,N,NaN
2,201801,70071840.001.000000066,AC,1,BBS,EAXMXMFSRN1,202,01.10.2019 00:00:00,30.09.2021 00:00:00,24.0,20.0,290.0,10M,NaN,NaN,0,Y,N,N,NaN


In [368]:
# View the data
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510125 entries, 0 to 510124
Data columns (total 20 columns):
image                 510125 non-null int64
newacct_no            510125 non-null object
line_stat             510125 non-null object
bill_cycl             510125 non-null int64
serv_type             510125 non-null object
serv_code             510125 non-null object
tenure                510125 non-null int64
effc_strt_date        508188 non-null object
effc_end_date         508188 non-null object
contract_month        508188 non-null float64
ce_expiry             508188 non-null float64
secured_revenue       510125 non-null float64
bandwidth             510125 non-null object
term_reas_code        5523 non-null object
term_reas_desc        5523 non-null object
complaint_cnt         510125 non-null object
with_phone_service    510125 non-null object
churn                 510125 non-null object
current_mth_churn     510125 non-null object
Unnamed: 19           2 non-null objec

In [369]:
# Check for duplicate rows
churn.duplicated().any()

False

In [370]:
# Check for empty/null rows
churn.dropna(how='all', inplace=True)

In [371]:
# Extract 2019 from 'image' column
churn = churn[churn['image'] >= 201901]

In [372]:
# change the extract year and month out of the 'image' column
churn['image'] = churn['image'].astype('str')
churn['image']

247719    201901
247720    201901
247721    201901
247722    201901
247723    201901
           ...  
510120    201912
510121    201912
510122    201912
510123    201912
510124    201912
Name: image, Length: 262406, dtype: object

In [373]:
def date(value):
    if value.endswith('01'):
        return '2019-01'
    if value.endswith('02'):
        return '2019-02'
    if value.endswith('03'):
        return '2019-03'
    if value.endswith('04'):
        return '2019-04'
    if value.endswith('05'):
        return '2019-05'
    if value.endswith('06'):
        return '2019-06'
    if value.endswith('07'):
        return '2019-07'
    if value.endswith('08'):
        return '2019-08'
    if value.endswith('09'):
        return '2019-09'
    if value.endswith('10'):
        return '2019-10'
    if value.endswith('11'):
        return '2019-11'
    if value.endswith('12'):
        return '2019-12'
    

In [374]:
churn['year_month'] = churn['image'].apply(date)
churn.head(3)

,image,newacct_no,line_stat,bill_cycl,serv_type,serv_code,tenure,effc_strt_date,effc_end_date,contract_month,...,secured_revenue,bandwidth,term_reas_code,term_reas_desc,complaint_cnt,with_phone_service,churn,current_mth_churn,Unnamed: 19,year_month
247719,201901,70068143.001.000000062,AC,1,BBS,PF500DUN,214,06.03.2018 00:00:00,05.03.2021 00:00:00,36.0,...,248.0,500M (FTTO),NaN,NaN,0,N,N,N,NaN,2019-01
247720,201901,70068143.003.000072630,AC,1,BBS,VS030F6R,47,01.03.2019 00:00:00,28.02.2021 00:00:00,24.0,...,1572.0,30M,NaN,NaN,0,N,N,N,NaN,2019-01
247721,201901,70068143.004.000089578,AC,1,BBS,PF1000FSH,7,09.07.2018 00:00:00,08.07.2020 00:00:00,24.0,...,488.0,1000M (FTTO),NaN,NaN,0,N,N,N,NaN,2019-01


In [375]:
# Convert year_date column to datetime and to period
churn['year_month'] = pd.to_datetime(churn['year_month'],yearfirst=False)
churn['year_month']

247719   2019-01-01
247720   2019-01-01
247721   2019-01-01
247722   2019-01-01
247723   2019-01-01
            ...    
510120   2019-12-01
510121   2019-12-01
510122   2019-12-01
510123   2019-12-01
510124   2019-12-01
Name: year_month, Length: 262406, dtype: datetime64[ns]

In [376]:
# Lets groupby year_month
date = churn.groupby(['year_month', 'churn'])
date['churn'].count()

year_month  churn
2019-01-01  N        18635
            Y         2876
2019-02-01  N        18863
            Y         2709
2019-03-01  N        19156
            Y         2518
2019-04-01  N        19419
            Y         2270
2019-05-01  N        19750
            Y         2043
2019-06-01  N        20080
            Y         1777
2019-07-01  N        20405
            Y         1570
2019-08-01  N        20682
            Y         1297
2019-09-01  N        21005
            Y         1059
2019-10-01  N        21279
            Y          803
2019-11-01  N        21575
            Y          517
2019-12-01  N        21873
            Y          245
Name: churn, dtype: int64

### The month of January 2019 experienced the most Defectors numbering  2876, with customer churn declining overtime.

In [377]:
# Lets groupby bandwith
bandw = churn.groupby(['bandwidth', 'churn'])
bandw['churn'].count()

bandwidth     churn
1000M (FTTO)  N         18379
              Y           727
100M          N        125929
              Y          9750
100M (FTTO)   N         45684
              Y          3311
10M           N         22931
              Y          2993
300M (FTTO)   N          6547
              Y           306
30M           N         18876
              Y          2191
500M (FTTO)   N          2363
              Y           186
50M           N          1003
              Y            83
BELOW 10M     N          1010
              Y           137
Name: churn, dtype: int64

### The 100M Bandwidth has the most custormer churn rate of  9750 customers leaving.

In [378]:
# Export the modified data as csv
churn.to_csv('modified_churn.csv', encoding='utf-8', index=False)